In [97]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [98]:
response = requests.get('http://www.metrolyrics.com/rem-lyrics.html')
doc = BeautifulSoup(response.text)

In [99]:
table = doc.select_one('#popular > div > table > tbody').find_all('tr')

rows = []
for item in table:
    row = {}
    row['title']=(item.a.text.strip())
    row['link']=(item.a['href'])
    row['year']=(item.find_all('td')[2].text)
    row['popularity']=(item.find_all('td')[3].span['class'][1])
    rows.append(row)

rows

[{'title': "It's The End Of The World Lyrics",
  'link': 'http://www.metrolyrics.com/its-the-end-of-the-world-lyrics-rem.html',
  'year': '2008',
  'popularity': 'popular10'},
 {'title': 'Losing My Religion Lyrics',
  'link': 'http://www.metrolyrics.com/losing-my-religion-lyrics-rem.html',
  'year': '2003',
  'popularity': 'popular9'},
 {'title': 'The One I Love Lyrics',
  'link': 'http://www.metrolyrics.com/the-one-i-love-lyrics-rem.html',
  'year': '2008',
  'popularity': 'popular9'},
 {'title': 'Selingkar Kasih Lyrics',
  'link': 'http://www.metrolyrics.com/selingkar-kasih-lyrics-rem.html',
  'year': '2010',
  'popularity': 'popular7'},
 {'title': 'Man On The Moon Lyrics',
  'link': 'http://www.metrolyrics.com/man-on-the-moon-lyrics-rem.html',
  'year': '2003',
  'popularity': 'popular7'},
 {'title': 'The Sidewinder Sleeps Tonight Lyrics',
  'link': 'http://www.metrolyrics.com/the-sidewinder-sleeps-tonight-lyrics-rem.html',
  'year': '2006',
  'popularity': 'popular7'},
 {'title': '

In [100]:
df = pd.DataFrame(rows)

In [101]:
df.popularity = df.popularity.replace(to_replace=r'popular', value='', regex=True)

In [102]:
df.to_csv("lyrics.csv")

In [103]:
df.head()

,link,popularity,title,year
0,http://www.metrolyrics.com/its-the-end-of-the-...,10,It's The End Of The World Lyrics,2008
1,http://www.metrolyrics.com/losing-my-religion-...,9,Losing My Religion Lyrics,2003
2,http://www.metrolyrics.com/the-one-i-love-lyri...,9,The One I Love Lyrics,2008
3,http://www.metrolyrics.com/selingkar-kasih-lyr...,7,Selingkar Kasih Lyrics,2010
4,http://www.metrolyrics.com/man-on-the-moon-lyr...,7,Man On The Moon Lyrics,2003


In [75]:
# url = "http://www.metrolyrics.com/its-the-end-of-the-world-lyrics-rem.html"
# response = requests.get(url)
# doc = BeautifulSoup(response.text)

# page = {}
# lyrics = doc.find_all('p', class_="verse")
# page['lyrics'] = '\n'.join([lyric.text for lyric in lyrics])



In [104]:
def scrape_page(row):
    url = row['link']
#     print("Scraping", url)
    
    response = requests.get(url)
    doc = BeautifulSoup(response.text)
    
    page = {}
    lyrics = doc.find_all('p', class_="verse")
    page['lyrics'] = '\n'.join([lyric.text for lyric in lyrics])
    
    return pd.Series(page)

In [105]:
scraped_df = df.apply(scrape_page, axis=1)

In [106]:
scraped_df = scraped_df[scraped_df.lyrics != '']

In [107]:
df = df.merge(scraped_df, left_index=True, right_index=True)

In [109]:
df.head(15)

,link,popularity,title,year,lyrics
0,http://www.metrolyrics.com/its-the-end-of-the-...,10,It's The End Of The World Lyrics,2008,"That's great, it starts with an earthquake\nBi..."
1,http://www.metrolyrics.com/losing-my-religion-...,9,Losing My Religion Lyrics,2003,"Oh life, it's bigger\nIt's bigger than you\nAn..."
2,http://www.metrolyrics.com/the-one-i-love-lyri...,9,The One I Love Lyrics,2008,This one goes out to the one I love\nThis one ...
3,http://www.metrolyrics.com/selingkar-kasih-lyr...,7,Selingkar Kasih Lyrics,2010,Seharusnya cinta kita bisa tersenyum\nSeolah s...
4,http://www.metrolyrics.com/man-on-the-moon-lyr...,7,Man On The Moon Lyrics,2003,"and the game of Life\nYeah, yeah, yeah, yeah\n..."
5,http://www.metrolyrics.com/the-sidewinder-slee...,7,The Sidewinder Sleeps Tonight Lyrics,2006,This here is the place i will be staying.\nthe...
6,http://www.metrolyrics.com/nightswimming-lyric...,6,Nightswimming Lyrics,2003,Nightswimming deserves a quiet night\nThe phot...
7,http://www.metrolyrics.com/everybody-hurts-lyr...,6,Everybody Hurts Lyrics,2003,"When your day is long\nAnd the night, the nigh..."
8,http://www.metrolyrics.com/fall-on-me-lyrics-r...,6,Fall On Me Lyrics,2011,"There's a problem, feathers, iron\nBargain bui..."
9,http://www.metrolyrics.com/orange-crush-lyrics...,5,Orange Crush Lyrics,2003,"(Follow me, don't follow me)\nI've got my spin..."


In [110]:
df.to_csv("lyrics_final.csv", index=False)